In [ ]:
# !pip install kaggle
# from google.colab import files

# # Upload Kaggle API key (kaggle.json)
# files.upload()

# # Move kaggle.json to the correct directory
# !mkdir -p ~/.kaggle
# !mv kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# !kaggle datasets download -d sanikatiwarekar/deep-fake-detection-dfd-entire-original-dataset
# !unzip deep-fake-detection-dfd-entire-original-dataset.zip -d /content/dataset/videos

In [ ]:
# !kaggle datasets download -d manjilkarki/deepfake-and-real-images
# !unzip deepfake-and-real-images.zip -d /content/dataset/images

In [ ]:
import os

print("Video Dataset Files:", os.listdir("/content/dataset/videos")[:10])
print("Image Dataset Files:", os.listdir("/content/dataset/images")[:10])

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# تحديد المسارات
train_dir = "/content/dataset/images/Dataset/Train"
test_dir = "/content/dataset/images/Dataset/Test"

# تهيئة مولد البيانات مع إعادة تحجيم الصور
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# تحميل البيانات للتدريب
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="training"
)

# تحميل بيانات التحقق
val_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)

# تحميل بيانات الاختبار
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary"
)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

# إنشاء نموذج CNN محسّن
model = Sequential([
    Input(shape=(224, 224, 3)),  # الطريقة الموصى بها
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # تصنيف ثنائي (Fake أو Real)
])

# تجميع النموذج
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# عرض ملخص النموذج
model.summary()

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10  # يمكنك زيادة عدد الصور لتحسين الدقة
)

In [ ]:
# تقييم النموذج على بيانات الاختبار
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
model.save("/content/deepfake_model.h5")

In [ ]:
import IPython.display as display

# تشغيل فيديو معين
output_path='/content/dataset/videos/DFD_original sequences'
video_files = os.listdir(output_path)
print("Available video files:", video_files[:100])  # عرض أول 10 فيديوهات فقط
video_path = os.path.join(output_path, video_files[0])
display.Video(video_path)


In [ ]:
import cv2

def extract_frames(video_path, output_folder, num_frames=100):
    cap = cv2.VideoCapture(video_path)
    count = 0

    while cap.isOpened() and count < num_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(output_folder, f"frame_{count}.jpg")
        cv2.imwrite(frame_path, frame)
        count += 1

    cap.release()
    print(f"Extracted {count} frames from {video_path}")

# إنشاء مجلد للإطارات المستخرجة
frames_output_path = "/content/extracted_frames"
os.makedirs(frames_output_path, exist_ok=True)

# استخراج الإطارات من فيديو اختبار
extract_frames(video_path, frames_output_path)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# تحميل النموذج المدرب
model = tf.keras.models.load_model("/content/deepfake_model.h5")

# التنبؤ بالفيديو باستخدام أول إطار مستخرج
test_frame = os.path.join(frames_output_path, "frame_0.jpg")
img = image.load_img(test_frame, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# التنبؤ
prediction = model.predict(img_array)[0][0]
print("Prediction:", "Fake" if prediction > 0.5 else "Real")